# ボートレーサーごとの統計量をdfに追加　その１
- 各枠からの1着率, 2着率, 3着率をそれぞれ計算し、特徴量として追加

## 1. 統計解析用のdfをロード
- `race_df`に格納
- 2_データ前処理.ipnbを参照

In [1]:
import pandas as pd
race_df = pd.read_pickle('../../data/arrangedData/data_formatted_for_statistic.pkl').reset_index(drop=True)
# 中身を確認
race_df.head()

,date,venue,raceNumber,枠,racer_id,racer_class,racer_name,num_false_start,num_late_start,motorNo,...,exhibition_cource,exhibition_ST,flying,late,着順,タイム,start_time,weather_曇り,weather_雨,weather_雪
0,2020-11-01,下 関,1.0,1,4060,2,島田 一生,0,0,35,...,1,0.05,0.0,0,1,"1'48""4",0.11,1,0,0
1,2020-11-01,下 関,1.0,2,3680,3,森林 太,0,0,37,...,2,0.11,0.0,0,2,"1'50""8",0.17,1,0,0
2,2020-11-01,下 関,1.0,3,3123,3,中西 宏文,0,0,66,...,3,0.18,0.0,0,5,,0.09,1,0,0
3,2020-11-01,下 関,1.0,4,5001,3,坂本 雅佳,0,0,20,...,4,0.01,0.0,0,4,"1'53""9",0.14,1,0,0
4,2020-11-01,下 関,1.0,5,4219,3,本岡 勝利,0,0,60,...,5,0.21,0.0,0,3,"1'52""1",0.17,1,0,0


## 2. これまでのレース結果から統計量を算出
- 各選手が、その枠からの1着率・２着率・3着率を算出し、それぞれ列に格納
- 選手ごと、枠ごと、日付ごとに、その前日までのレースを対象に統計量を算出

In [2]:
from tqdm.notebook import tqdm

def calc_rank_i_ratio(data, rank_i):
    # rank_i着であった率を算出
    total_count = data.count()
    rank_i_count = (data == rank_i).sum()
    rank_i_ratio = rank_i_count / total_count
    
    return rank_i_ratio


date_list = sorted(race_df["date"].unique())
date_list = date_list[1:]

stocastic_df_list = []

for date in tqdm(date_list):

    the_date_df_list = []
    
    for i in range(1,4):
        race_df_before_the_date = race_df[race_df["date"] < date]

        # その日までのデータを集めたdfに対して、選手ごとにi着率を算出
        stocastic_df_ = race_df_before_the_date.groupby(["racer_id", "枠"])["着順"].apply(
            calc_rank_i_ratio, rank_i=i).reset_index()
        stocastic_df_.rename(columns={"着順": "{0}着率".format(i)}, inplace=True)
        
        # racer_idと枠をmultiindexとして指定
        stocastic_df_.set_index(["racer_id", "枠"], inplace=True)
        
        the_date_df_list.append(stocastic_df_)
    
    # その日までのレーサーごとの統計量を横方向に結合
    stocastic_df_date = pd.concat(the_date_df_list, axis=1)

    # dfにdate列を追加
    stocastic_df_date["date"] = date
    # その日についてまとめたデータをリストに格納
    stocastic_df_list.append(stocastic_df_date)

# あらゆる日付について縦方向にマージ
stocastic_df = pd.concat(stocastic_df_list)
# indexを振り直す
stocastic_df.reset_index(inplace=True)

stocastic_df

  0%|          | 0/119 [00:00<?, ?it/s]

KeyboardInterrupt: 

## 3. `race_df`に2で算出した統計量をマージ

In [ ]:
merge_key_list = ["date", "枠", "racer_id"]
race_df_w_stocastic = pd.merge(race_df, stocastic_df,
                               left_on=merge_key_list, right_on=merge_key_list,
                               how="left"
                              )
race_df_w_stocastic

## 4. 3.で作った統計量入りのdfをpickleファイルにして保存

In [ ]:
race_df_w_stocastic.to_pickle('../../data/arrangedData/race_df_w_stocastic_2.pkl')